# 标准Softmax数值稳定性 - 实践篇

本notebook通过实际代码帮助你理解Softmax的数值稳定性问题。

**学习目标：**
- 观察直接计算Softmax的数值溢出问题
- 实现数值稳定的Softmax
- 验证减去最大值技巧的正确性


## 环境准备


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"NumPy版本: {np.__version__}")
print(f"PyTorch版本: {torch.__version__}")


## 1. Softmax基础


In [ ]:
def softmax_formula(x):
    """
    Softmax公式: softmax(x)_i = exp(x_i) / sum(exp(x_j))
    
    这是直接按定义实现的版本，有数值稳定性问题！
    """
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# 测试正常情况
x = np.array([1.0, 2.0, 3.0])
result = softmax_formula(x)

print("输入:", x)
print("Softmax输出:", result)
print("输出之和:", np.sum(result))
print("\n✓ 正常情况下工作正常")


## 2. 数值溢出问题


In [ ]:
# 测试大数情况
print("=" * 50)
print("测试大数情况")
print("=" * 50)

x_large = np.array([1000.0, 1001.0, 1002.0])

print(f"\n输入: {x_large}")
print(f"exp(1000) = {np.exp(1000.0)}")
print(f"exp(1001) = {np.exp(1001.0)}")
print(f"exp(1002) = {np.exp(1002.0)}")

result_large = softmax_formula(x_large)
print(f"\nSoftmax输出: {result_large}")
print("\n✗ 输出是NaN，计算失败！")


In [ ]:
# 测试小数情况（负数）
print("=" * 50)
print("测试极小数情况")
print("=" * 50)

x_small = np.array([-1000.0, -1001.0, -1002.0])

print(f"\n输入: {x_small}")
print(f"exp(-1000) = {np.exp(-1000.0)}")
print(f"exp(-1001) = {np.exp(-1001.0)}")

result_small = softmax_formula(x_small)
print(f"\nSoftmax输出: {result_small}")
print("\n✗ 输出是NaN（0/0的结果）")


## 3. 数值稳定的Softmax


In [ ]:
def stable_softmax(x):
    """
    数值稳定的Softmax实现
    
    关键技巧：减去最大值
    softmax(x - max(x)) = softmax(x)
    """
    m = np.max(x)           # 找最大值
    exp_x = np.exp(x - m)   # 减去最大值后再exp
    l = np.sum(exp_x)       # 求和
    return exp_x / l

# 验证数学等价性
print("=" * 50)
print("验证：减去最大值不改变结果")
print("=" * 50)

x = np.array([1.0, 2.0, 3.0])

result_naive = softmax_formula(x)
result_stable = stable_softmax(x)

print(f"\n输入: {x}")
print(f"直接计算: {result_naive}")
print(f"稳定版本: {result_stable}")
print(f"差异: {np.max(np.abs(result_naive - result_stable))}")
print("\n✓ 结果完全一致！")


In [ ]:
# 测试稳定版本处理大数
print("=" * 50)
print("稳定版本处理大数")
print("=" * 50)

x_large = np.array([1000.0, 1001.0, 1002.0])

print(f"\n输入: {x_large}")
print(f"最大值: {np.max(x_large)}")
print(f"减去最大值后: {x_large - np.max(x_large)}")

result_stable = stable_softmax(x_large)
print(f"\nSoftmax输出: {result_stable}")
print(f"输出之和: {np.sum(result_stable)}")
print("\n✓ 正确计算！")

# 与小数版本对比
x_small_equiv = np.array([0.0, 1.0, 2.0])  # [1000, 1001, 1002] - 1000
print(f"\n对比 [0, 1, 2] 的结果: {stable_softmax(x_small_equiv)}")
print("✓ 结果一致，验证平移不变性")


## 4. 可视化对比


In [ ]:
# 可视化减去最大值的效果
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

x = np.linspace(-5, 5, 100)
y = np.exp(x)

# 图1: exp函数
axes[0].plot(x, y, 'b-', linewidth=2)
axes[0].axhline(y=1, color='r', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='r', linestyle='--', alpha=0.5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('exp(x)')
axes[0].set_title('指数函数 exp(x)')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 50])

# 图2: 原始值的exp
x_orig = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
exp_orig = np.exp(x_orig)
axes[1].bar(range(len(x_orig)), exp_orig, color='steelblue', alpha=0.7)
axes[1].set_xlabel('索引')
axes[1].set_ylabel('exp(x)')
axes[1].set_title('原始值的exp')
for i, v in enumerate(exp_orig):
    axes[1].text(i, v + 2, f'{v:.1f}', ha='center')

# 图3: 减去最大值后的exp
x_shifted = x_orig - np.max(x_orig)
exp_shifted = np.exp(x_shifted)
axes[2].bar(range(len(x_shifted)), exp_shifted, color='coral', alpha=0.7)
axes[2].set_xlabel('索引')
axes[2].set_ylabel('exp(x - max)')
axes[2].set_title('减去最大值后的exp')
for i, v in enumerate(exp_shifted):
    axes[2].text(i, v + 0.02, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.show()

print("关键：减去最大值后，所有exp值都 ≤ 1，不会溢出！")


## 5. 与PyTorch对比验证


In [ ]:
# 与PyTorch的实现对比
print("=" * 50)
print("与PyTorch实现对比")
print("=" * 50)

test_cases = [
    np.array([1.0, 2.0, 3.0]),
    np.array([100.0, 200.0, 300.0]),
    np.array([-1.0, 0.0, 1.0]),
    np.random.randn(10) * 100,
]

for i, x in enumerate(test_cases):
    x_torch = torch.tensor(x, dtype=torch.float32)
    
    our_result = stable_softmax(x)
    torch_result = F.softmax(x_torch, dim=0).numpy()
    
    max_diff = np.max(np.abs(our_result - torch_result))
    
    print(f"\n测试 {i+1}:")
    print(f"  输入范围: [{x.min():.2f}, {x.max():.2f}]")
    print(f"  最大差异: {max_diff:.2e}")
    print(f"  ✓ 通过" if max_diff < 1e-6 else f"  ✗ 失败")


## 6. 分块处理的问题


In [ ]:
# 演示分块处理的问题
print("=" * 50)
print("分块处理的问题")
print("=" * 50)

# 假设有一个长序列
full_sequence = np.array([1.0, 3.0, 2.0, 5.0, 4.0, 6.0])
print(f"\n完整序列: {full_sequence}")
print(f"正确的Softmax结果: {stable_softmax(full_sequence)}")

# 分成两块处理
block1 = full_sequence[:3]  # [1, 3, 2]
block2 = full_sequence[3:]  # [5, 4, 6]

print(f"\n分块:")
print(f"  Block 1: {block1}, 局部最大值 = {np.max(block1)}")
print(f"  Block 2: {block2}, 局部最大值 = {np.max(block2)}")

# 如果用局部最大值计算
result1_wrong = stable_softmax(block1)
result2_wrong = stable_softmax(block2)

print(f"\n如果独立处理每个块:")
print(f"  Block 1的Softmax: {result1_wrong}")
print(f"  Block 2的Softmax: {result2_wrong}")

# 拼接结果
wrong_result = np.concatenate([result1_wrong, result2_wrong])
print(f"\n拼接结果: {wrong_result}")
print(f"拼接结果之和: {np.sum(wrong_result):.4f}")
print("\n✗ 问题：和不等于1，结果错误！")
print("\n原因：每个块使用了不同的局部最大值和局部归一化因子")
print("\n解决方案：需要Online Softmax算法来正确合并各块的结果！")


## 7. 总结

### 关键点

1. **直接计算Softmax存在数值溢出风险**
   - 大正数 → exp上溢为inf
   - 大负数 → exp下溢为0
   - 导致NaN结果

2. **解决方案：减去最大值**
   - 利用Softmax的平移不变性
   - 保证所有exp值在[0, 1]范围内

3. **标准实现需要三遍遍历**
   - Pass 1: 找最大值
   - Pass 2: 计算归一化因子
   - Pass 3: 计算输出

4. **分块处理的挑战**
   - 无法预知全局最大值
   - 独立处理各块会导致错误结果
   - 需要Online Softmax来解决

### 下一步

学习 **Online Softmax**，了解如何在只看到部分数据的情况下正确计算Softmax！
